### Imports

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import trainer_lib as tl

torch.manual_seed(310231551)
random.seed(3009231410)
np.random.seed(2909231846)
np_random_state = np.random.RandomState(131002)

### Load data

In [2]:
df: pd.DataFrame = tl.load_country_wide_dataset('../data/country_data.csv')

X = df.to_numpy(dtype=np.float32)
y = df['el_load'].to_numpy(dtype=np.float32)

print(X.shape)

(75960, 11)


### Define models

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, features=11, hidden_size=15, num_layers=2, bidirectional=True):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.h_n_dim = 2 if bidirectional else 1
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=features, hidden_size=self.hidden_size, num_layers=num_layers, batch_first=True, bidirectional=bidirectional)
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.hidden_size * self.h_n_dim * self.num_layers, 3)
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h_0 = torch.zeros(self.h_n_dim * self.num_layers, batch_size, self.hidden_size).requires_grad_().to(tl.TRAINER_LIB_DEVICE)
        c_0 = torch.zeros(self.h_n_dim * self.num_layers, batch_size, self.hidden_size).requires_grad_().to(tl.TRAINER_LIB_DEVICE)

        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        h_n = torch.permute(h_n, (1, 0, 2)) # From shape [h_n_dim, batch, hidden_size] -> [batch, h_n_dim, hidden_size]
                                            # flatten and fully connected layer expects batch to be the first dimension
        return self.fc(h_n)

### Grid search

I'll first look at different model constructions, then I'll look into hyperparameters, dropouts, noise and maybe higher sequence lengths.

In [11]:
grid = tl.Grid({
    'epochs': [1000],  # we use early stopping, so this is just a high number
    'lr': [0.001],
    'hidden_size': [15, 30],
    'num_layers': [1, 2],
    'bidirectional': [False, True],
    'n_splits': [6],
})

wrapper = tl.MIMOTSWrapper(LSTMModel(), seq_len=24, pred_len=3)
b_p, b_s = wrapper.grid_search(X, y, grid, verbose=4)
print(f"Best params: {b_p}\nBest score: {b_s}")

[Grid search 001] BEGIN - params: {'epochs': 1000, 'lr': 0.001, 'hidden_size': 15, 'num_layers': 1, 'bidirectional': False, 'n_splits': 6}
[Fold 1] BEGIN
Early stopping... Epoch 152: train loss: 0.003689, val loss: 0.048384, test loss: 0.045673
[Fold 1] END - RMSE loss: 146.754 - Time: 1.5 min.
[Fold 2] BEGIN
Early stopping... Epoch 031: train loss: 0.009497, val loss: 0.028678, test loss: 0.030876
[Fold 2] END - RMSE loss: 102.948 - Time: 0.5 min.
[Fold 3] BEGIN
Early stopping... Epoch 084: train loss: 0.004522, val loss: 0.017538, test loss: 0.007031
[Fold 3] END - RMSE loss: 60.133 - Time: 1.9 min.
[Fold 4] BEGIN
Early stopping... Epoch 095: train loss: 0.004587, val loss: 0.019779, test loss: 0.016760
[Fold 4] END - RMSE loss: 92.231 - Time: 2.8 min.
[Fold 5] BEGIN
Early stopping... Epoch 032: train loss: 0.007409, val loss: 0.025110, test loss: 0.023704
[Fold 5] END - RMSE loss: 111.279 - Time: 1.2 min.
[Fold 6] BEGIN
Early stopping... Epoch 043: train loss: 0.007017, val loss: 0.